<center>

#     
### NATURAL LANGUAGE PROCESSING
## 2023./2024.
# Word Embeddings
## Data: hrWaC, ParlaSent
#### Nika Hell i Josipa Radak      

<img src="https://drive.google.com/uc?export=view&id=1brAAaBxs7E3ZZi2-rwm5K5E1lKp-pgWh" width="600" height="600">

</center>

##     

After preparing data and training the model, this journey came to the testing step. Here we will test our models and see how good they perform. We will look at several metrics for model testing:

* **Accuracy**: Accuracy measures the proportion of correct predictions among the total number of predictions made by the model. It is a simple and intuitive measure of the model's performance.

* **Precision**: Precision measures the proportion of true positive predictions among all positive predictions made by the model. It indicates how many of the predicted positive instances are actually positive.

* **Recall**: Recall, also known as sensitivity or true positive rate, measures the proportion of true positive predictions among all actual positive instances in the data. It indicates how many of the actual positive instances were correctly predicted by the model.

<center>

<img src="https://www.researchgate.net/publication/358507092/figure/fig6/AS:1152919537762304@1651888846152/Accuracy-precision-and-recall-equations.png" width="400" height="200">

</center>

* **F1-score**: F1-score is the harmonic mean of precision and recall. It provides a balance between precision and recall and is often used as a single metric to evaluate classification models.

<center>

<img src="https://images.prismic.io/encord/0ef9c82f-2857-446e-918d-5f654b9d9133_Screenshot+%2849%29.png?auto=compress,format" width="240" height="80">

</center>

**Content of the notebook**

> Libraries and arguments

> Connecting to Google Drive

> Helpers

>> Model class

>> Loading helpers

> Model evaluation

>> Load the data

>> Load the model

>> Prepare data

>> Evaluate model and calculate metrics

> Classifier based on GloVe

>> Load the data

>> Load the model

>> Prepare data

>> Model

>> Evaluate model and calculate metrics


## Libraries and arguments

In [ ]:
import pickle  # For serializing Python objects
import torch  # PyTorch library for machine learning

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix  # Metrics for evaluating classification performance
from sklearn.preprocessing import LabelEncoder  # For encoding categorical labels into numerical format
from sklearn.model_selection import train_test_split  # For splitting data into training and testing sets
from sklearn.preprocessing import StandardScaler  # For feature scaling
from keras.layers import Dense  # Dense layer for fully connected neural networks
from keras.models import Sequential  # Sequential model for stacking layers sequentially

import numpy as np  # import NumPy for scientific computing
import os  # import the os module for operating system functionalities
import pandas as pd  # import Pandas for data manipulation and analysis
import csv  # import the csv module for reading and writing CSV files

import torch.nn as nn  # import the nn module from PyTorch for building neural network layers and architectures
import torch.nn.functional as F  # import the functional interface of PyTorch for neural network operations
import torch.optim as optim  # import the optim module from PyTorch for optimization algorithms such as SGD, Adam, etc.
from torch.utils.data import Dataset, DataLoader  # import Dataset and DataLoader for handling data loading in PyTorch
from torch.nn.utils.rnn import pad_sequence  # import pad_sequence function for padding variable-length sequences
from torch.optim.lr_scheduler import ReduceLROnPlateau  # import ReduceLROnPlateau for adjusting learning rate during training
from torch.nn.functional import cross_entropy  # import cross_entropy function for calculating cross-entropy loss

from argparse import Namespace # the namespace class is used to create an object that holds attributes corresponding to command-line arguments

To avoid having arguments in multiple places, we add a Namespace to store all necessary arguments. This is important for better organization and readability of the code, as it allows us to group related arguments together and access them easily throughout the program. Additionally, using a Namespace helps prevent naming conflicts and makes it simpler to pass arguments between different parts of the code.

In [ ]:
args = Namespace(
    vocabulary="/content/drive/MyDrive/nlp_project/preprocessing_glove/glove_vocab.pkl",  # path to the file where the vocabulary is saved
    co_occurrence_matrix_test = '/content/drive/MyDrive/nlp_project/preprocessing_glove/co_occurrence_matrix_test.pkl',
    checkpoint_path1="glove_checkpoint.pth",  # path to the checkpoint of the model
    classificator_train='/content/drive/MyDrive/nlp_project/preprocessing/classificator_train.csv',
    classificator_test='/content/drive/MyDrive/nlp_project/preprocessing/classificator_test.csv',
)

## Connecting to Google Drive

In this section, we are connecting to Google Drive in order to store the necessary files on disk as well as retrieve the required files from the disk. Furthermore, to keep everything organized, we have decided to store all outputs resulting from this notebook in a single directory named "preprocessing". Additionally, there is already a file named 'nlp.py' from which we will extract the necessary classes for some necessary steps.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/nlp_project/GloVe

/content/drive/.shortcut-targets-by-id/1dadLNnejsKWL1qhI8AmHtxjCR430s42m/nlp_project/GloVe


## Helpers

### Model class

In [ ]:
class GloveModel(nn.Module):
  def __init__(self, vocab_size, embedding_dim):
    """
    Initialize the Glove model.
    Args:
        vocab_size (int): Size of the vocabulary.
        embedding_dim (int): Dimensionality of word embeddings.
    """
    super(GloveModel, self).__init__()
    # define embedding layers for target and context words
    self.embedding_target = nn.Embedding(vocab_size, embedding_dim)
    self.embedding_context = nn.Embedding(vocab_size, embedding_dim)
    # define bias embeddings for target and context words
    self.bias_target = nn.Embedding(vocab_size, 1)
    self.bias_context = nn.Embedding(vocab_size, 1)

  def forward(self, target, context):
    """
    Forward pass of the Glove model.
    Args:
        target (torch.Tensor): Indices of target words.
        context (torch.Tensor): Indices of context words.
    Returns:
        torch.Tensor: The result of the forward pass.
    """
    # embed target and context words
    emb_target = self.embedding_target(target)
    emb_context = self.embedding_context(context)
    # get bias embeddings for target and context words
    bias_target = self.bias_target(target).squeeze(1)
    bias_context = self.bias_context(context).squeeze(1)
    # compute dot product between target and context word embeddings
    dot_product = torch.sum(emb_target * emb_context, dim=1)
    # return the sum of dot product and biases
    return dot_product + bias_target + bias_context

  def cosine_similarity(self, embedding1, embedding2):
    """
    Calculate cosine similarity between two embeddings.
    Args:
        embedding1 (torch.Tensor): Embedding of the first vector.
        embedding2 (torch.Tensor): Embedding of the second vector.
    Returns:
        torch.Tensor: Cosine similarity between the two embeddings.
    """
    # normalize embeddings
    embedding1_norm = F.normalize(embedding1, p=2, dim=-1)
    embedding2_norm = F.normalize(embedding2, p=2, dim=-1)
    # calculate dot product
    dot_product = torch.sum(embedding1_norm * embedding2_norm, dim=-1)
    return dot_product

### Loading helpers

In [ ]:
# attempts to open a specified file in binary read mode
# uses the pickle module to deserialize and load the vocabulary from the file
# returns the loaded vocabulary
# handles the case where the file is not found or encounters reading errors, printing appropriate error messages
def load_serialized_object(filename='serialized_data.pkl'):
  # attempt to open the specified file in binary read ('rb') mode
  try:
    with open(filename, 'rb') as file:
      # use the pickle module to deserialize and load the data from the file
      loaded_data = pickle.load(file)
    # return the loaded data
    return loaded_data
  # handle the case where the file is not found
  except FileNotFoundError:
    # print an error message indicating that the file was not found
    print(f"Error: File '{filename}' not found.")
    # return None to signify the absence of data
    return None
  # handle other exceptions, such as reading errors
  except Exception as e:
    # print an error message with details about the encountered exception
    print(f"Error loading data from '{filename}': {e}")
    # return None to signify the inability to load the data
    return None

In [ ]:
# function definition for loading a checkpoint
def load_model(model, optimizer, filename):
  # check the availability of GPU and set the appropriate device
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  # check if the checkpoint file exists
  if os.path.exists(filename):
    # if the file exists, load the checkpoint
    checkpoint = torch.load(filename)
    # load model's state, optimizer's state, epoch, and train loss from the checkpoint
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # move the model to the appropriate device
    model.to(device)
    # return the loaded model, optimizer, epoch, and train loss
    return model, optimizer
  # if the file doesn't exist, raise a FileNotFoundError
  else:
    raise FileNotFoundError("Checkpoint file does not exist.")

## Model evaluation

### Load the data

In [ ]:
glove_vocab = load_serialized_object(args.vocabulary)
co_occurrence_matrix_test = load_serialized_object(args.co_occurrence_matrix_test)
df_train = pd.read_csv(args.classificator_train)
df_test = pd.read_csv(args.classificator_test)

###Load the model

In [ ]:
# initialize the GloveModel with the specified vocabulary size and embedding dimension
model = GloveModel(len(glove_vocab), embedding_dim=200)

# define the loss function to be Mean Squared Error (MSE)
# loss_fn = nn.MSELoss()
loss_fn =  nn.BCEWithLogitsLoss()
# Initialize the Adam optimizer with a learning rate,
# optimizing the parameters of the model
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model, optimizer= load_model(model, optimizer, args.checkpoint_path1)

### Prepare data

In [ ]:
# initialize an empty list to store test data
test_data = []

# iterate over each target index and corresponding context dictionary in the co-occurrence matrix for the test data
for target_idx, context_dict in co_occurrence_matrix_test.items():
  # iterate over each context index and count in the context dictionary
  for context_idx, count in context_dict.items():
    # add a tuple (target_idx, context_idx, count) to the test data list
    test_data.append((target_idx, context_idx, count))

In [ ]:
test_data[:60]

[(291838, 364619, 146),
 (291838, 763066, 719),
 (291838, 410194, 1),
 (291838, 369193, 692),
 (291838, 155482, 3),
 (291838, 822167, 1),
 (291838, 341822, 377),
 (291838, 778209, 1),
 (291838, 123879, 15),
 (291838, 635797, 1),
 (291838, 88168, 113),
 (291838, 196805, 3),
 (291838, 748007, 3),
 (291838, 802177, 11),
 (291838, 551502, 148),
 (291838, 421393, 1),
 (291838, 647727, 220),
 (291838, 428528, 32),
 (291838, 373779, 12),
 (291838, 737129, 1),
 (291838, 833612, 1),
 (291838, 552260, 1),
 (291838, 47941, 3),
 (291838, 820558, 3),
 (291838, 221742, 5),
 (291838, 105454, 1),
 (291838, 333401, 107),
 (291838, 286571, 1),
 (291838, 460235, 38),
 (291838, 469991, 2),
 (291838, 837397, 9),
 (291838, 345927, 19),
 (291838, 708388, 3),
 (291838, 224175, 1),
 (291838, 517852, 54),
 (291838, 176823, 1),
 (291838, 617619, 2),
 (291838, 672512, 10),
 (291838, 157352, 5),
 (291838, 375636, 2),
 (291838, 839038, 1),
 (291838, 756476, 32),
 (291838, 135270, 7),
 (291838, 0, 95),
 (291838, 438

In [ ]:
vocab_size = len(glove_vocab) # size of vocabulary

### Evaluate model and calculate metrics

For our test dataset, we have extracted words from sentences. For each word, we considered other words within a 2-word window as context words. We then paired each target word with its context words and calculated how many times they co-occurred together in the dataset. In this test, we aim to determine whether the presence of contextual words can predict the association with the target word. If the probability of association is greater than 50%, the label is set to true. Otherwise, it is set to false.

To evaluate the model's performance, we set it to evaluation mode and initialized lists to store predictions and labels. Then, we iterated through the test data, disabling gradient calculation for inference. For each target and context pair, we conducted a forward pass to obtain logits from the model. These logits were transformed into binary predictions using the sigmoid function, and the label was set to true if the co-occurrence count was greater than 0, otherwise false

In [ ]:
# Selecting the appropriate device for computation based on GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Set the model to evaluation mode
model.eval()

# Initialize lists to store predictions and labels
all_predictions = []
all_labels = []

# sisable gradient calculation for inference
with torch.no_grad():
    # iterate through the test data
    for target_idx, context_idx, count in test_data:
        # move target and context indices to the specified device (CPU or GPU)
        target_idx_tensor = torch.tensor([target_idx]).to(device)
        context_idx_tensor = torch.tensor([context_idx]).to(device)

        # forward pass to get logits from the model
        logits = model(target_idx_tensor, context_idx_tensor)

        # convert logits to binary predictions using sigmoid function
        prediction = (torch.sigmoid(logits) > 0.5).item()

        # set label to True if the co-occurrence count is greater than 0, otherwise False
        label = count > 0

        # append prediction and label to the lists
        all_predictions.append(prediction)
        all_labels.append(label)

In [ ]:
# Calculating evaluation metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions)
recall = recall_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions)

print("Accuracy on test data:", round(accuracy * 100, 2), '%')
print("Precision on test data:", round(precision* 100, 2), '%')
print("Recall on test data:", round(recall* 100, 2), '%')
print("F1 score on test data:", round(f1* 100, 2), '%')

Accuracy on test data: 94.13 %
Precision on test data: 100.0 %
Recall on test data: 94.13 %
F1 score on test data: 96.98 %


The evaluation metrics on the test data indicate strong performance: an accuracy of 94.13%, precision of 100.0%, recall of 94.13%, and an F1 score of 96.98%. These results suggest that our model effectively predicts associations between target and contextual words based on their co-occurrence patterns.

## Classifier based on GloVe

### Load the data

In [ ]:
glove_vocab = load_serialized_object(args.vocabulary)
df_train = pd.read_csv(args.classificator_train)
df_test = pd.read_csv(args.classificator_test)

### Load the model

In [ ]:
# initialize the GloveModel with the specified vocabulary size and embedding dimension
model_glove = GloveModel(len(glove_vocab), embedding_dim=200)

# define the loss function to be Mean Squared Error (MSE)
loss_fn_glove = nn.BCEWithLogitsLoss()
# nn.MSELoss()

# Initialize the Adam optimizer with a learning rate,
# optimizing the parameters of the model
optimizer_glove = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model_glove, optimizer_glove= load_model(model_glove, optimizer_glove, args.checkpoint_path1)

### Prepare data

In [ ]:
df_train.head()

,sentence,label
0,da li je pošteno da se ukida prethodna stopa i...,Negative
1,znam pouzdano da su među specijalnim snagama b...,Negative
2,vizija predstavlja sliku idealne budućnosti a ...,Other
3,znači banke imaju 9 i pol milijardi kuna slobo...,Other
4,oni mogu biti i oružana agresija dakle napad d...,Negative


In [ ]:
df_test.head()

,sentence,label
0,nesporno je da bi svi u državi srbiji želeli d...,Negative
1,ti plasmani porasli su više nego što su plasma...,Other
2,metodologija je vrlo jednostavna grupa stručnj...,Other
3,jer podsjetit ću da nepotizam definicijom znač...,Negative
4,dalje utvrđeno je da bi to bila da je to najva...,Other


In [ ]:
def embed_sentences(df, glove_vocab, model, device):
  embedded_sentences = []  # Array for storing embedded sentences

  # iterate through the DataFrame
  for index, row in df.iterrows():
      sentence = row['sentence']
      label = row['label']

      # tokenization of the sentence
      tokens = sentence.split(' ')

      # initialize a list to store word embeddings in the sentence
      sentence_embeddings = []

      # convert words to vectors
      for token in tokens:
        if token in glove_vocab:
          # retrieve vector for the word from the GloVe model
          word_vector = model.embedding_target(torch.tensor([glove_vocab[token]], device=device)).detach().cpu().numpy()
          # word_vector = model.embedding_target(torch.tensor([glove_vocab[token]], device=device)).detach().numpy()
          # add the vector to the list of word embeddings in the sentence
          sentence_embeddings.append(word_vector)

      # calculate sentence embedding
      if len(sentence_embeddings) > 0:
        # calculate the average vector of all words in the sentence
        sentence_embedding = np.mean(sentence_embeddings, axis=0)
        # add the sentence embedding to the list of embedded sentences along with the label
        embedded_sentences.append((sentence_embedding, label))

  # convert the list to a DataFrame
  embedded_df = pd.DataFrame(embedded_sentences, columns=['embedded_sentence', 'label'])

  return embedded_df

In [ ]:
embedded_df_train = embed_sentences(df_train, glove_vocab, model_glove, 'cuda')
embedded_df_test = embed_sentences(df_test, glove_vocab, model_glove, 'cuda')

In [ ]:
embedded_df_train.head()

,embedded_sentence,label
0,"[[0.073219076, 0.5645261, -0.029062131, -0.293...",Negative
1,"[[-0.004341119, 0.38779926, 0.04984711, -0.307...",Negative
2,"[[0.52013195, 0.13883796, -0.31362683, -0.4501...",Other
3,"[[0.47545287, 0.61617464, -0.12192415, -0.3104...",Other
4,"[[0.52715605, -0.03423869, -0.09515445, 0.0971...",Negative


In [ ]:
# Podaci
X_train = np.vstack(embedded_df_train['embedded_sentence'].to_numpy())
y_train = embedded_df_train['label'].to_numpy()
X_test = np.vstack(embedded_df_test['embedded_sentence'].to_numpy())
y_test = embedded_df_test['label'].to_numpy()

# Inicijalizacija LabelEncoder-a
label_encoder = LabelEncoder()

# Pretvaranje tekstualnih oznaka u numeričke vrijednosti
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
# Converting training data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)  # Features tensor
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.float32).squeeze()  # Labels tensor, squeezing to remove extra dimensions if present


In [ ]:
# Converting testing data to PyTorch tensors
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)  # Features tensor
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.float32).squeeze()  # Labels tensor, squeezing to remove extra dimensions if present

### Model

In [ ]:
# Define the Sequential model
cls = Sequential()

# Add layers to the model
cls.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))  # Input layer with ReLU activation
cls.add(Dense(64, activation='relu'))  # Hidden layer with ReLU activation
cls.add(Dense(1, activation='sigmoid'))  # Output layer with sigmoid activation for binary classification

# Compile the model
cls.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Using Adam optimizer and binary crossentropy loss

# Train the model
cls.fit(X_train, y_train_encoded, epochs=100, batch_size=512, verbose=1, validation_split=0.2)  # Training the model for 100 epochs with batch size


Epoch 1/100
5/5 [==============================] - 1s 50ms/step - loss: 0.6933 - accuracy: 0.5212 - val_loss: 0.6894 - val_accuracy: 0.5462
Epoch 2/100
5/5 [==============================] - 0s 11ms/step - loss: 0.6699 - accuracy: 0.5784 - val_loss: 0.6847 - val_accuracy: 0.5481
Epoch 3/100
5/5 [==============================] - 0s 11ms/step - loss: 0.6611 - accuracy: 0.5990 - val_loss: 0.6783 - val_accuracy: 0.5481
Epoch 4/100
5/5 [==============================] - 0s 10ms/step - loss: 0.6402 - accuracy: 0.6481 - val_loss: 0.6852 - val_accuracy: 0.5654
Epoch 5/100
5/5 [==============================] - 0s 14ms/step - loss: 0.6316 - accuracy: 0.6490 - val_loss: 0.6799 - val_accuracy: 0.5423
Epoch 6/100
5/5 [==============================] - 0s 15ms/step - loss: 0.6177 - accuracy: 0.6788 - val_loss: 0.6813 - val_accuracy: 0.5615
Epoch 7/100
5/5 [==============================] - 0s 10ms/step - loss: 0.6081 - accuracy: 0.6861 - val_loss: 0.6905 - val_accuracy: 0.5558
Epoch 8/100
5/5 [===

### Evaluate model and calculate metrics

In [ ]:
# make predictions
y_pred_proba = cls.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int)

# classification score
print(classification_report(y_test_encoded, y_pred))

# accuracy
accuracy = accuracy_score(y_test_encoded, y_pred)
print("Točnost klasifikatora:", round(accuracy, 2)*100, '%')

# F1 score
f1 = f1_score(y_test_encoded, y_pred, average='weighted')
print("F1 mjera:", round(f1, 2)*100, '%')

82/82 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.54      0.60      0.57      1147
           1       0.65      0.60      0.63      1453

    accuracy                           0.60      2600
   macro avg       0.60      0.60      0.60      2600
weighted avg       0.61      0.60      0.60      2600

Točnost klasifikatora: 60.0 %
F1 mjera: 60.0 %


The classifier achieved an accuracy of 60.0%. Precision varied between 54% for class 0 and 65% for class 1, while recall was consistent at 60% for both classes. The F1-score for class 0 was 0.57, and for class 1, it was 0.63. These metrics collectively indicate moderate performance across both classes. The macro and weighted averages provide a comprehensive assessment of the classifier's performance across all classes. Overall, the classifier demonstrates a fair ability to distinguish between the two classes.

### Without embedding

In [ ]:
df_train.head()

,sentence,label
0,da li je pošteno da se ukida prethodna stopa i...,Negative
1,znam pouzdano da su među specijalnim snagama b...,Negative
2,vizija predstavlja sliku idealne budućnosti a ...,Other
3,znači banke imaju 9 i pol milijardi kuna slobo...,Other
4,oni mogu biti i oružana agresija dakle napad d...,Negative


In [ ]:
df_test.head()

,sentence,label
0,nesporno je da bi svi u državi srbiji želeli d...,Negative
1,ti plasmani porasli su više nego što su plasma...,Other
2,metodologija je vrlo jednostavna grupa stručnj...,Other
3,jer podsjetit ću da nepotizam definicijom znač...,Negative
4,dalje utvrđeno je da bi to bila da je to najva...,Other


In [ ]:
# TF-IDF vectorization
vectorizer = TfidfVectorizer()
X_train_sparse = vectorizer.fit_transform(df_train['sentence'])
X_test_sparse = vectorizer.transform(df_test['sentence'])

# Converting sparse matrix to CSR format
X_train_csr = csr_matrix(X_train_sparse)
X_test_csr = csr_matrix(X_test_sparse)

# Label encoding
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_train['label'])
y_test = label_encoder.transform(df_test['label'])

# Converting to tensors
X_train_tensor = torch.tensor(X_train_csr.toarray(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_csr.toarray(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.int64)
y_test_tensor = torch.tensor(y_test, dtype=torch.int64)

In [ ]:
# Data standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_tensor)
X_test_scaled = scaler.transform(X_test_tensor)

# Model definition
cls2 = Sequential()
cls2.add(Dense(512, activation='relu', input_shape=(X_train_scaled.shape[1],)))
cls2.add(Dense(64, activation='relu'))
cls2.add(Dense(1, activation='sigmoid'))

# Model compilation
cls2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Convert target tensors to NumPy arrays
y_train_numpy = y_train_tensor.numpy()

# Model training
cls2.fit(X_train_scaled, y_train_numpy, epochs=100, batch_size=512, verbose=1)

Epoch 1/100
6/6 [==============================] - 1s 19ms/step - loss: 0.7370 - accuracy: 0.5573
Epoch 2/100
6/6 [==============================] - 0s 22ms/step - loss: 0.0932 - accuracy: 0.9854
Epoch 3/100
6/6 [==============================] - 0s 21ms/step - loss: 0.0191 - accuracy: 1.0000
Epoch 4/100
6/6 [==============================] - 0s 19ms/step - loss: 0.0072 - accuracy: 1.0000
Epoch 5/100
6/6 [==============================] - 0s 19ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 6/100
6/6 [==============================] - 0s 20ms/step - loss: 9.3336e-04 - accuracy: 1.0000
Epoch 7/100
6/6 [==============================] - 0s 18ms/step - loss: 5.5406e-04 - accuracy: 1.0000
Epoch 8/100
6/6 [==============================] - 0s 19ms/step - loss: 3.8575e-04 - accuracy: 1.0000
Epoch 9/100
6/6 [==============================] - 0s 19ms/step - loss: 2.9851e-04 - accuracy: 1.0000
Epoch 10/100
6/6 [==============================] - 0s 19ms/step - loss: 2.4200e-04 - accuracy: 1.0000

In [ ]:
# Make predictions
y_pred_proba2 = cls2.predict(X_test_scaled)
y_pred2 = (y_pred_proba2 > 0.5).astype(int)

# Convert predictions to NumPy array
y_pred2_numpy = np.squeeze(y_pred2)

# Classification score
print(classification_report(y_test, y_pred2_numpy))

# Accuracy
accuracy = accuracy_score(y_test, y_pred2_numpy)
print("Accuracy:", round(accuracy, 2) * 100, '%')

# F1 score
f1 = f1_score(y_test, y_pred2_numpy, average='weighted')
print("F1 score:", round(f1, 2) * 100, '%')

82/82 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.58      0.57      0.57      1147
           1       0.66      0.67      0.67      1453

    accuracy                           0.63      2600
   macro avg       0.62      0.62      0.62      2600
weighted avg       0.63      0.63      0.63      2600

Accuracy: 63.0 %
F1 score: 63.0 %


The classification report offers a comprehensive assessment of the model's performance. For the negative class (class 0), the precision, recall, and F1-score are 0.58, 0.57, and 0.57, respectively. This indicates that while 58% of the instances classified as negative are indeed negative (precision), the model correctly identifies 57% of the actual negative instances (recall). The F1-score, which balances precision and recall, is also 0.57. Conversely, for the positive class (class 1), the precision, recall, and F1-score are 0.66, 0.67, and 0.67, respectively. This suggests that the model performs slightly better in identifying positive instances, with 66% precision and 67% recall. The overall accuracy of the model stands at 63%, indicating that 63% of the instances are correctly classified. Both the macro average and weighted average F1-scores are 0.62, highlighting the model's consistent performance across both classes. In conclusion, the model demonstrates decent performance, with balanced precision, recall, and F1-scores for both classes.